In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

DATADIR = "/content/drive/My Drive/PseudoDataTrain"

CATEGORIES = ["type_1", "type_2", "type_3"]

from google.colab import drive
drive.mount('/content/drive')


In [0]:
training_data = []
IMG_SIZE = 450

def create_training_data():
  for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)  
    class_num = CATEGORIES.index(category)  # if 0, type 1 data; 1, type 2 data; 2, type 3 data. 
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)  # convert to array
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])

create_training_data()

print(len(training_data))

In [0]:
DATADIR_VAL = "/content/drive/My Drive/PseudoDataValid"

validation_data = []

def create_validation_data():
  for category in CATEGORIES:  
    path = os.path.join(DATADIR_VAL,category)  
    class_num = CATEGORIES.index(category)  # if 0, type 1 data; 1, type 2 data; 3, type 3 data. 
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)  # convert to array
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        validation_data.append([new_array, class_num])

create_validation_data()

print(len(validation_data))

In [0]:
import random 

random.shuffle(training_data)
random.shuffle(validation_data)


In [0]:
X = []
y = []

X_val = []
y_val = []


In [0]:
for features, label in training_data:
  X.append(features)
  y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

for features, label in validation_data:
  X_val.append(features)
  y_val.append(label)

X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)




In [0]:
import pickle

pickle_out = open('X.pickle', 'wb')
pickle.dump(X, pickle_out)  # dumps X into pickle_out
pickle_out.close()

pickle_out = open('y.pickle', 'wb')
pickle.dump(y, pickle_out)  # dumps y into pickle_out
pickle_out.close()

pickle_out = open('X_val.pickle', 'wb')
pickle.dump(X_val, pickle_out)  # dumps X_val into pickle_out
pickle_out.close()

pickle_out = open('y_val.pickle', 'wb')
pickle.dump(y_val, pickle_out)  # dumps y_val into pickle_out
pickle_out.close()


In [0]:
from tensorflow.keras import layers, models
import pickle

train_images = pickle.load(open('X.pickle', 'rb'))  # the data
train_labels = pickle.load(open('y.pickle', 'rb'))  # labels for each data

test_images = pickle.load(open('X_val.pickle', 'rb'))  # the data
test_labels = pickle.load(open('y_val.pickle', 'rb'))  # labels for each data

train_images, test_images = train_images / 255.0, test_images / 255.0


In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.summary()


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=5, 
                    validation_data=(test_images, test_labels))

In [0]:
DATADIR_EVAL = "/content/drive/My Drive/PseudoDataReal"

evaluation_data = []

def create_evaluation_data():
  for category in CATEGORIES:  
    path = os.path.join(DATADIR_EVAL,category)  
    class_num = CATEGORIES.index(category)  # if 0, type 1 data; 1, type 2 data; 2, type 3 data. 
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)  # convert to array
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        evaluation_data.append([new_array, class_num])

create_evaluation_data()

print(len(evaluation_data))


In [0]:
X_eval = []
y_eval = []

In [0]:
random.shuffle(evaluation_data)

for features, label in evaluation_data:
  X_eval.append(features)
  y_eval.append(label)

X_eval = np.array(X_eval).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_eval = X_eval/255.0


In [0]:
model.evaluate(X_eval,  y_eval, verbose=1)